In [173]:
en_path = '/Users/liuchu/linear_algebra_strang/translate/中英翻译数据集/train/news-commentary-v13.zh-en.en'
zh_path = '/Users/liuchu/linear_algebra_strang/translate/中英翻译数据集/train/news-commentary-v13.zh-en.zh'

In [174]:
def get_en_sentences():
    arr = []
    with open(en_path,'r') as f:
        count = 0
        for line in f.readlines():
            arr.append(line.strip())
    return arr

In [175]:
def get_zh_sentences():
    arr = []
    with open(zh_path,'r') as f:
        count = 0
        for line in f.readlines():
            arr.append(line.strip())
    return arr

In [176]:
en_sentences = get_en_sentences()
zh_sentences = get_zh_sentences()

In [178]:
src_sentences = en_sentences
target_sentences = zh_sentences

In [153]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 编码器
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# 解码器
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = nn.functional.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [142]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

    decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS_token是起始符号的标记

    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == EOS_token:  # EOS_token是结束符号的标记
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [154]:
def tensorFromSentence(vocab, sentence):
    indices = [vocab[char] for char in sentence if char in vocab]  # Ensure char is in vocab
    indices.append(vocab['<eos>'])  # Append EOS token
    return torch.tensor(indices, dtype=torch.long, device=device).view(-1, 1)

def translate_sentence(sentence, encoder, decoder, input_vocab, output_vocab):
    input_tensor = tensorFromSentence(input_vocab, sentence)
    translated_sentence = translate(encoder, decoder, input_tensor, input_vocab, output_vocab)
    return translated_sentence

def translate(encoder, decoder, input_tensor, input_vocab, output_vocab, max_length=10):
    with torch.no_grad():
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.initHidden()

        # Forward pass through encoder
        for ei in range(input_length):
            _, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        # Decoder's first input is SOS token
        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_word = output_vocab[topi.item()]
                decoded_words.append(decoded_word)

            decoder_input = topi.squeeze().detach()

        return ' '.join(decoded_words)

In [179]:
# 假设语料数据如下，这里我们使用了非常简单的例子
chinese_sentences = zh_sentences
english_sentences = en_sentences

# 为了简化问题，我们不使用词嵌入，而是直接根据字母或汉字进行索引
def create_dict(sentences):
    vocab = set(''.join(sentences))
    char_to_index = {char: i+1 for i, char in enumerate(vocab)}  # +1因为我们留出0作为padding
    char_to_index['<pad>'] = 0
    index_to_char = {i: char for char, i in char_to_index.items()}
    return char_to_index, index_to_char

chinese_vocab, chinese_index_vocab = create_dict(chinese_sentences)
chinese_vocab['<sos>'] = max(chinese_vocab.values()) + 1
chinese_index_vocab[chinese_vocab['<sos>']] = '<sos>'

english_vocab, english_index_vocab = create_dict(english_sentences)

# 将句子转换为索引
def sentence_to_indices(sentence, vocab):
    return [vocab[char] for char in sentence]

chinese_data = [sentence_to_indices(sentence, chinese_vocab) for sentence in chinese_sentences]
english_data = [sentence_to_indices(sentence, english_vocab) for sentence in english_sentences]

# 增加开始和结束标记

SOS_token = english_vocab['<sos>'] = max(english_vocab.values()) + 1
EOS_token = english_vocab['<eos>'] = max(english_vocab.values()) + 1
english_index_vocab[SOS_token] = '<sos>'
english_index_vocab[EOS_token] = '<eos>'
english_data = [[SOS_token] + sentence + [EOS_token] for sentence in english_data]

In [180]:
len(english_index_vocab),len(english_vocab)

(194, 194)

In [182]:
len(chinese_vocab)

4647

In [183]:
# 将数据转换为torch张量
def to_tensor(data):
    return torch.tensor(data, dtype=torch.long, device=device).view(-1, 1)

chinese_tensors = [to_tensor(data) for data in chinese_data]
english_tensors = [to_tensor(data) for data in english_data]

# 训练参数设置
num_epochs = 100
learning_rate = 0.01
encoder = Encoder(len(chinese_vocab), 256).to(device)
decoder = Decoder(256, len(english_vocab)).to(device)
encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()



In [ ]:
# 训练循环
for epoch in range(num_epochs):
    total_loss = 0
    for ch_tensor, en_tensor in zip(chinese_tensors, english_tensors):
        loss = train(ch_tensor, en_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(chinese_sentences):.4f}')

In [164]:
def translate_sentence(sentence):
    indices = sentence_to_indices(sentence, chinese_vocab)
    tensor = to_tensor(indices)
    translation = translate(encoder, decoder, tensor, chinese_index_vocab, english_index_vocab)
    return translation

# 翻译示例
print(translate_sentence('我'))
print(translate_sentence('我是学生'))

<sos> i   l i k e   s t
<sos> i   a m   a   s t
